In [1]:
import sys
import os
import numpy as np
import pandas as pd
import re
import itertools
import tensorflow as tf
import string
from io import BytesIO
from tensorflow.contrib import learn
from collections import Counter
from time import time
import datetime
from sklearn import model_selection
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder

In [2]:
d = pd.read_csv("C:/Users/student/Downloads/consumer_complaints2.csv", 
                usecols=('product','consumer_complaint_narrative'),
                dtype={'consumer_complaint_narrative': object})
# Only interested in data with consumer complaints
d=d[d['consumer_complaint_narrative'].notnull()]
d=d[d['product'].notnull()]
d.reset_index(drop=True,inplace=True)

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
text = []
text = d['consumer_complaint_narrative']
# create the transform
vectorizer = TfidfVectorizer()
# tokenize and build vocab
vectorizer.fit(text)
# summarize
print(vectorizer.vocabulary_)
print(vectorizer.idf_)
# encode document
vector = vectorizer.transform([text[0]])
# summarize encoded vector
#print(vector.shape)
#print(vector.toarray())

{'xxxx': 52619, 'has': 22342, 'claimed': 10036, 'owe': 33742, 'them': 46682, '27': 827, '00': 0, 'for': 20111, 'years': 53834, 'despite': 14742, 'the': 46468, 'proof': 36693, 'of': 32492, 'payment': 34313, 'sent': 42029, 'canceled': 8681, 'check': 9634, 'and': 4134, 'their': 46625, 'ownpaid': 33786, 'invoice': 25993, 'they': 46982, 'continue': 12106, 'to': 47666, 'insist': 25252, 'collection': 10534, 'agencies': 3239, 'are': 5076, 'after': 3145, 'me': 29085, 'how': 23347, 'can': 8667, 'stop': 44606, 'this': 47115, 'harassment': 22236, 'bill': 7257, 'already': 3729, 'paid': 33847, 'four': 20518, 'ago': 3336, 'due': 16577, 'inconsistencies': 24394, 'in': 24147, 'amount': 4013, 'owed': 33743, 'that': 46385, 'was': 51245, 'told': 47809, 'by': 8370, 'comcast': 10667, 'reported': 39460, 'billing': 7279, 'issues': 26236, 'advised': 2971, 'write': 52469, 'good': 21550, 'will': 51980, 'letter': 27573, 'order': 33167, 'address': 2736, 'issue': 26217, 'request': 39741, 'negative': 31280, 'entry':

In [4]:
def clean_str(string):
    """
    Tokenization/string cleaning (partially modified)
    """
    string = re.sub(r"[^A-Za-z0-9()!?\'\`%$]", " ", string) # keep also %$ but removed comma
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " ( ", string)
    string = re.sub(r"\)", " ) ", string)
    string = re.sub(r"\?", " ? ", string)
    string = re.sub(r"\$", " $ ", string) #yes, isolate $
    string = re.sub(r"\%", " % ", string) #yes, isolate %
    string = re.sub(r"\s{2,}", " ", string)
    
    # fixing XXX and xxx like as word
    string = re.sub(r'\S*(x{2,}|X{2,})\S*',"xxx",string)
    # removing non ascii
    string = re.sub(r'[^\x00-\x7F]+', "", string) 
    
    return string.strip().lower()

In [5]:
word_data=[]
t0 = time()

for message in d['consumer_complaint_narrative']:
    word_data.append(clean_str(message))

max_document_length = max([len(x.split(" ")) for x in word_data])
print ("Max_document_length:",max_document_length)
vocab_processor = learn.preprocessing.VocabularyProcessor(max_document_length)
num_data = np.array(list(vocab_processor.fit_transform(word_data)))
print("Vocabulary Size: {:d}".format(len(vocab_processor.vocabulary_)))

np.random.seed(57)
shuffle_indices = np.random.permutation(np.arange(len(num_data)))
x_shuffled = num_data[shuffle_indices]
y_shuffled = d['product'][shuffle_indices]
print ("* x shuffled:", x_shuffled.shape)
print ("* y shuffled:", y_shuffled.shape)

features_dummy, x_test, labels_dummy, test_labels = model_selection.train_test_split(x_shuffled, y_shuffled, test_size=0.20, random_state= 23)
x_train, x_valid, train_labels, valid_labels = model_selection.train_test_split(features_dummy, labels_dummy, test_size=0.25, random_state= 34)

print('Training set  ',   x_train.shape, train_labels.shape)
print('Validation set',   x_valid.shape, valid_labels.shape)
print('Test set      ',    x_test.shape,  test_labels.shape)

# free some memory
del num_data, d 
del x_shuffled, y_shuffled, labels_dummy, features_dummy



def oneHot(dummy_labels):
    le = preprocessing.LabelEncoder()
    enc = OneHotEncoder()
    
    le.fit (dummy_labels)
    y_dummy = le.fit_transform(dummy_labels)
    y_dummy = y_dummy.reshape(-1, 1)
    enc.fit(y_dummy)
    y_dummy = enc.transform(y_dummy).toarray()
    y_dummy = y_dummy.astype('float32')
    print ("\n * OneHot example")
    print (y_dummy)
    return (y_dummy)
        
y_train = oneHot(train_labels)
y_valid = oneHot(valid_labels)
y_test  = oneHot( test_labels)

Max_document_length: 910
Vocabulary Size: 52925
* x shuffled: (66806, 910)
* y shuffled: (66806,)
Training set   (40083, 910) (40083,)
Validation set (13361, 910) (13361,)
Test set       (13362, 910) (13362,)

 * OneHot example
[[ 1.  0.  0. ...,  0.  0.  0.]
 [ 1.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 1.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]

 * OneHot example
[[ 0.  0.  0. ...,  0.  0.  1.]
 [ 1.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 1.  0.  0. ...,  0.  0.  0.]
 [ 1.  0.  0. ...,  0.  0.  0.]
 [ 1.  0.  0. ...,  0.  0.  0.]]

 * OneHot example
[[ 1.  0.  0. ...,  0.  0.  0.]
 [ 1.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  1. ...,  0.  0.  0.]]


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(x_train, y_train) 
pred = neigh.predict(x_test)

In [ ]:
print(neigh.score(x_test,y_test))